In [1]:
import pandas as pd
import os
from pathlib import Path
from tqdm.notebook import tqdm

In [2]:
data = {}

for file in tqdm(os.listdir("../../data/parquet")):
    file_path = os.path.join("../../data/parquet", file)
    year = int(Path(file_path).stem)
    data[year] = pd.read_parquet(file_path)

  0%|          | 0/11 [00:00<?, ?it/s]

In [3]:
fiscal_year = lambda year: (
    pd.Timestamp(year=year - 1, month=7, day=1),
    pd.Timestamp(year=year, month=6, day=30),
)

filter_row = lambda beginning, ending: lambda row: (
    beginning < pd.Timestamp(row["Issue Date"], unit="ms")
) and (pd.Timestamp(row["Issue Date"], unit="ms") < ending)


def filter_rows_by_date(dataframe, year):
    fiscal_row_filter = filter_row(*fiscal_year(year))
    return dataframe.apply(fiscal_row_filter, axis=1)

`9100275` Rows before filtering by fiscal dates for the 2014 dataset, and `9087141` after filtering for the fiscal year.

In [4]:
for year in tqdm(data.keys()):
    data[year] = data[year][filter_rows_by_date(data[year], year)]

  0%|          | 0/11 [00:00<?, ?it/s]

In [5]:
import numpy as np

column_naming_convention = {}
column_typing_convention = {}

for year in data.keys():
    for i, column in enumerate(data[year].columns):
        if i not in column_naming_convention:
            column_naming_convention[i] = []
        column_naming_convention[i].append(column)

    for i, dtype in enumerate(data[year].dtypes):
        if i not in column_typing_convention:
            column_typing_convention[i] = []
        column_typing_convention[i].append(dtype)

for column in column_naming_convention:
    values, counts = np.unique(column_naming_convention[column], return_counts=True)
    column_naming_convention[column] = (values, counts)

for column in column_typing_convention:
    values, counts = np.unique(column_typing_convention[column], return_counts=True)
    column_typing_convention[column] = (values, counts)

column_names = [
    counting_data[0][np.argmax(counting_data[1])].strip()
    for _, counting_data in column_naming_convention.items()
]

column_types = [
    counting_data[0][np.argmax(counting_data[1])]
    for _, counting_data in column_typing_convention.items()
]

for year in data.keys():
    data[year].columns = column_names

type_dict = {
    column: dtype for column, dtype in zip(column_names, column_types)
}

In [6]:
def hand_curated_type_definitions(data, column):
    if column in ["Vehicle Expiration Date", "Date First Observed"]:
        data[column] = (
            pd.to_numeric(data[column].replace("01/05/0001 12:00:00 PM", ""))
            .fillna(-1)
            .astype(np.int64)
        )
    elif column in ["Vehicle Year", "Violation Location"]:
        data[column] = pd.to_numeric(data[column]).fillna(-1).astype(np.int64)
    else:
        raise ValueError(f"Column {column} is not supported")
    return data

type_dict["Unregistered Vehicle?"] = np.dtype("O")

for year in data.keys():
    parsed = False
    while not parsed:
        try:
            data[year] = data[year].astype(type_dict)
            parsed = True
        except Exception as e:
            error_column = str(e).split("column")[-1].strip().replace("'", "")
            print(f"Error in year {year}: {e}")
            data[year] = hand_curated_type_definitions(data[year], error_column)

Error in year 2014: invalid literal for int() with base 10: '': Error while type casting for column 'Violation Location'
Error in year 2014: invalid literal for int() with base 10: '': Error while type casting for column 'Vehicle Year'
Error in year 2015: invalid literal for int() with base 10: '': Error while type casting for column 'Vehicle Expiration Date'
Error in year 2015: invalid literal for int() with base 10: '': Error while type casting for column 'Violation Location'
Error in year 2015: invalid literal for int() with base 10: '01/05/0001 12:00:00 PM': Error while type casting for column 'Date First Observed'
Error in year 2015: invalid literal for int() with base 10: '': Error while type casting for column 'Vehicle Year'
Error in year 2016: invalid literal for int() with base 10: '': Error while type casting for column 'Violation Location'
Error in year 2016: invalid literal for int() with base 10: '': Error while type casting for column 'Vehicle Year'
Error in year 2017: in

In [7]:
data_concatenated = pd.concat([data[year] for year in data.keys()])

In [25]:
highest_column_frequency = {}

In [27]:
for column in data_concatenated.columns:
    counts = data_concatenated[column].value_counts()
    highest_column_frequency[column] = (
        (counts.index, counts.values / len(data_concatenated))
    )

In [53]:
nullable_values = [-1, ""]
columns_to_remove = []

In [ ]:
for column in highest_column_frequency:
    values = highest_column_frequency[column][0]
    frequencies = highest_column_frequency[column][1]
    frequencies_of_nullable_values = np.array([(values[i], frequencies[i]) for i in range(len(values)) if values[i] in nullable_values])
    if frequencies_of_nullable_values.size > 0 and (frequencies_of_nullable_values)[:,1].astype(np.float64).sum() > 0.75:
        columns_to_remove.append(column)

In [58]:
data_concatenated.drop(columns=columns_to_remove, inplace=True)